In [1]:
import networkx as nx
from functools import reduce


#############            Ancilary methods         #############
def in_neighbors(graph, v):
    return set([i for i,j in graph.edges if j == v]+[v])

def out_neighbors(graph, v):
    return set([j for i,j in graph.edges if i == v]+[v])

###############################################################


def distributed_graph_augmentation(graph, printFlag=False):
    classify = ['MIXED', 'TARGET', 'SOURCE', 'SOURCE_TARGET']
    n = graph.number_of_nodes()
    
    # - To change in an upgrade that considers edge costs - #
    weights=lambda x:1 
    total_weight = 0 
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - #
    
    DONE = False 
    n_rounds = 0
    while not DONE:
        n_rounds += 1
        if printFlag: print("=== Round",n_rounds,"===")
        neigh_in = [in_neighbors(graph, node) for node in range(n)]
        neigh_out = [out_neighbors(graph, node) for node in range(n)]
        x = [set([(i, j) for j in neigh_out[i] if j != i]) for i in range(n)]
        w = [False] * n
        s = [[] for i in range(n)]
        z = [{} for i in range(n)]


        # Distributed computation of SCCs and respective classification

        while not all(w):
            x_prev, w_prev = x.copy(), w.copy()
            for i in range(n):
                if not w[i]: # i did not reach a fixed point

                    # x[i] collects the edges forming paths that reach i
                    x[i] = set().union(*[x_prev[node] for node in neigh_in[i]])

                    # w[i] tracks if node i stops recieving new information
                    w[i] = len(x[i]) == len(x_prev[i]) 

        # SCCs reconstruction phase
        for i in range(n):            
            z[i] = set([a for (a, _) in x[i]]+[i]) # Nodes that reach i
            G = nx.DiGraph() # builds the upstream of i digraph
            G.add_edges_from(list(x[i]))
            #print(i,' ', x[i])

            # Computing SCCs using the obtained local information     
            s[i] = list(nx.strongly_connected_components(G))

            for j, scc in enumerate(s[i]):
                #out = z[i].difference(scc) # Nodes that reach i outside the scc

                # it is not a source if some node outside the scc reaches it
                not_source = [(u, v) for (u, v) in x[i] if u not in scc and v in scc] != [] 

                # it is not a target if some node outside the scc is reached by it
                not_target = [(u, v) for (u, v) in x[i] if u in scc and v not in scc] != []

                # type of scc 0 -> NEITHER, 1 -> TARGET, 2 -> SOURCE, 3 -> SOURCE_TARGET
                typ = not_source + 2 * not_target
                s[i][j] = (s[i][j], (classify[typ] if i in s[i][j] or not_target else '?'))

        # w.l.o.g. representative member of SCC is selected by default as the one with 
        # smaller id
        
        

        # target SCC nodes propose source SCC nodes
        proposals = {}

        if printFlag:
            print('SCCs:', [([el for el in z if el[-1] != '?']) for z in s] )
            print()
        
        # for the targets
        trgts_min_id = set(min(scc) for lst in s for (scc, typ) in lst if typ == 'TARGET')
        # NEW
        get_trgt_from_rep = {min(scc) : scc for lst in s for (scc, typ) in lst if typ == 'TARGET'}
        get_src_from_rep = {min(scc) : scc for lst in s for (scc, typ) in lst if typ == 'SOURCE'}
        # END NEW
   
        
        #trgts_set = set(min(s[i][j][0]) for (i, j) in trgts)
        src_reaching_trgt = {}
        src_reaching_trgt_total = {}
        #for (i, j) in trgts:
            #trgts_dic[min(s[i][j][0])] = [i] if j not in trgts_dic else trgts_dic[j] + [i] 
            #trgts_set.add()
          
        # A digraph with no targets is strongly connected (DONE)
        
        
        if trgts_min_id == set():
            DONE = True
            break

            

        #for tg in trgts_dic: 
        #for tg, src in [(i, s[i][j][0]) for i in trgts_min_id for j in range(len(s[i])) if s[i][j][1] == 'SOURCE']:
        #    src_reaching_trgt[tg] = [min(src)] if tg not in src_reaching_trgt else src_reaching_trgt[tg] + [min(src)]
        for i in trgts_min_id:
            src_reaching_trgt[i] = [min(scc) for (scc, typ) in s[i] if typ == 'SOURCE']
            # NEW
            src_reaching_trgt_total[i] = [scc for (scc, typ) in s[i] if typ == 'SOURCE']
            # END NEW
        
        if printFlag:
            print('TGT->SRC:', list(src_reaching_trgt.items()))
            # print('TGT IDs:',trgts_min_id)


        ############################################ THE PROPOSALS T-S ############################################
        
        src_received_props = {}
        for tg, src_lst in src_reaching_trgt.items():
            for i, rep in enumerate(src_lst):
                # NEW
                weight = min(weights((u, v)) for v in src_reaching_trgt_total[tg][i] for u in get_trgt_from_rep[tg])  
                #print(' >>>', tg+1, rep+1, [x+1 for x in src_reaching_trgt_total[tg][i]], \
                #                          [x+1 for x in get_trgt_from_rep[tg]])
                #print('-------- weight:', weight)
                if rep not in src_received_props:
                    src_received_props[rep] = [(tg, len(src_lst), weight)]  
                else:
                    src_received_props[rep].append((tg, len(src_lst), weight))  
                # END NEW
        if printFlag:
            print('SRC proposals:', list(src_received_props.items()))
            print('SRC->TGT:',list((s,min(s_lst, key=lambda z:(z[1]<=1, z[2]))[0]) for (s, s_lst) in src_received_props.items()))

        ############################################ THE PROPOSALS S-T ############################################

        target_sel = {}
        for (s, s_lst) in src_received_props.items():
            # sources strictly prefer targets that receive info from more than 1 source
            tmp = min(s_lst, key=lambda z:(z[1]<=1, z[2]))
            #print('escolha S:', tmp)
            t, weight = tmp[0], tmp[2]
            if t not in target_sel:
                target_sel[t] = [(s, len(src_received_props[s]), weight)]
            else:
                target_sel[t].append((s, len(src_received_props[s]), weight))
        
        
        if printFlag:
            print('TGT proposals:', list(target_sel.items()))
            print()

        ############################################## EDGE SELECTION ##############################################

        new_edges = []
        for (t, t_lst) in target_sel.items():
            # targets strictly prefer sources that send info to more than 1 target
            tmp = min(t_lst, key=lambda z:(z[1]<=1, z[2]))
            #print('escolha T:', tmp)
            s, weight = tmp[0], tmp[2]
            total_weight += weight
            new_edge = min([(u, v) for v in get_src_from_rep[s] \
                            for u in get_trgt_from_rep[t]], key=weights)
            new_edges.append(new_edge)
        
        if printFlag:
            print('New edges:', new_edges)
            print('Total weight:', total_weight)
            print()
            
        graph.add_edges_from(new_edges)
    return graph
 

In [3]:
edges4 = [(1,2),(2,1),(3,4),(4,3),(5,6),(5,7),(5,8),(6,5),(6,8),(7,6),(8,6),(8,7),(9,10),(10,11),(11,9),\
          (12,14),(13,12),(14,13),(1,11),(4,10),(4,13),(8,13)]
G4 = nx.DiGraph()
G4.add_edges_from([(i-1,j-1) for (i,j) in edges4])
result = distributed_graph_augmentation(G4, printFlag=True)
print([(i+1,j+1) for (i,j) in result.edges if (i+1,j+1) not in edges4])

=== Round 1 ===
SCCs: [[({0, 1}, 'SOURCE')], [({0, 1}, 'SOURCE')], [({2, 3}, 'SOURCE')], [({2, 3}, 'SOURCE')], [({4, 5, 6, 7}, 'SOURCE')], [({4, 5, 6, 7}, 'SOURCE')], [({4, 5, 6, 7}, 'SOURCE')], [({4, 5, 6, 7}, 'SOURCE')], [({8, 9, 10}, 'TARGET'), ({0, 1}, 'SOURCE'), ({2, 3}, 'SOURCE')], [({8, 9, 10}, 'TARGET'), ({0, 1}, 'SOURCE'), ({2, 3}, 'SOURCE')], [({8, 9, 10}, 'TARGET'), ({0, 1}, 'SOURCE'), ({2, 3}, 'SOURCE')], [({11, 12, 13}, 'TARGET'), ({4, 5, 6, 7}, 'SOURCE'), ({2, 3}, 'SOURCE')], [({11, 12, 13}, 'TARGET'), ({4, 5, 6, 7}, 'SOURCE'), ({2, 3}, 'SOURCE')], [({11, 12, 13}, 'TARGET'), ({4, 5, 6, 7}, 'SOURCE'), ({2, 3}, 'SOURCE')]]

TGT->SRC: [(8, [0, 2]), (11, [4, 2])]
SRC proposals: [(0, [(8, 2, 1)]), (2, [(8, 2, 1), (11, 2, 1)]), (4, [(11, 2, 1)])]
SRC->TGT: [(0, 8), (2, 8), (4, 11)]
TGT proposals: [(8, [(0, 1, 1), (2, 2, 1)]), (11, [(4, 1, 1)])]

New edges: [(8, 2), (11, 4)]
Total weight: 2

=== Round 2 ===
SCCs: [[({0, 1}, 'SOURCE')], [({0, 1}, 'SOURCE')], [({2, 3, 8, 9, 10}, '

In [5]:
edges5 = [(1, 2), (2, 3), (3, 2), (4, 5), (5, 4), (5, 6), (1, 6)]
G5 = nx.DiGraph()
G5.add_edges_from([(i-1,j-1) for (i,j) in edges5])
result = distributed_graph_augmentation(G5, printFlag=True)
print([(i+1,j+1) for (i,j) in result.edges if (i+1,j+1) not in edges4])

=== Round 1 ===
SCCs: [[({0}, 'SOURCE')], [({1, 2}, 'TARGET'), ({0}, 'SOURCE')], [({1, 2}, 'TARGET'), ({0}, 'SOURCE')], [({3, 4}, 'SOURCE')], [({3, 4}, 'SOURCE')], [({5}, 'TARGET'), ({0}, 'SOURCE'), ({3, 4}, 'SOURCE')]]

TGT->SRC: [(1, [0]), (5, [0, 3])]
SRC proposals: [(0, [(1, 1, 1), (5, 2, 1)]), (3, [(5, 2, 1)])]
SRC->TGT: [(0, 5), (3, 5)]
TGT proposals: [(5, [(0, 2, 1), (3, 1, 1)])]

New edges: [(5, 0)]
Total weight: 1

=== Round 2 ===
SCCs: [[({0, 5}, 'SOURCE_TARGET'), ({3, 4}, 'SOURCE')], [({1, 2}, 'TARGET'), ({0, 5}, 'SOURCE_TARGET'), ({3, 4}, 'SOURCE')], [({1, 2}, 'TARGET'), ({0, 5}, 'SOURCE_TARGET'), ({3, 4}, 'SOURCE')], [({3, 4}, 'SOURCE')], [({3, 4}, 'SOURCE')], [({0, 5}, 'SOURCE_TARGET'), ({3, 4}, 'SOURCE')]]

TGT->SRC: [(1, [3])]
SRC proposals: [(3, [(1, 1, 1)])]
SRC->TGT: [(3, 1)]
TGT proposals: [(1, [(3, 1, 1)])]

New edges: [(1, 3)]
Total weight: 2

=== Round 3 ===
SCCs: [[({0, 1, 2, 3, 4, 5}, 'MIXED')], [({0, 1, 2, 3, 4, 5}, 'MIXED')], [({0, 1, 2, 3, 4, 5}, 'MIXED')], 